<a href="https://colab.research.google.com/github/gaurav22m/Braille-to-English-Converter/blob/main/braille_resent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import sklearn
from tensorflow import keras
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.transforms import ToTensor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.datasets as datasets

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

# Function to load images from a folder
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images

# Function to preprocess images
def preprocess_images(images):
    # Resize images to a consistent size
    target_size = (100, 100)
    resized_images = resize_images(images, target_size)

    # Convert images to grayscale
    grayscale_images = convert_to_grayscale(resized_images)

    # Normalize pixel values
    normalized_images = normalize_images(grayscale_images)

    # Augment dataset if necessary
    augmented_images = augment_dataset(normalized_images)

    return augmented_images

# Function to resize images
def resize_images(images, target_size):
    resized_images = []
    for image in images:
        resized_image = cv2.resize(image, target_size)
        resized_images.append(resized_image)
    return resized_images

# Function to convert images to grayscale
def convert_to_grayscale(images):
    grayscale_images = []
    for image in images:
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        grayscale_images.append(grayscale_image)
    return grayscale_images

# Function to normalize pixel values
def normalize_images(images):
    normalized_images = []
    for image in images:
        normalized_image = image / 255.0  # Normalize pixel values to range [0, 1]
        normalized_images.append(normalized_image)
    return normalized_images

# Function to augment dataset
def augment_dataset(images):
    augmented_images = []
    for image in images:
        augmented_images.append(image)
    return augmented_images

# Path to the folder containing images
folder_path = '/content/drive/MyDrive/Braille Dataset2'

# Load images from each class folder
images_per_class = {}
for class_folder in os.listdir(folder_path):
    class_path = os.path.join(folder_path, class_folder)
    if os.path.isdir(class_path):
        images = load_images_from_folder(class_path)
        preprocessed_images = preprocess_images(images)
        images_per_class[class_folder] = preprocessed_images

# Now 'images_per_class' dictionary contains preprocessed images for each class

# Function to save preprocessed images to disk
def save_preprocessed_images(images, output_folder):
    for idx, image in enumerate(images):
        output_path = os.path.join(output_folder, f"image_{idx}.png")
        cv2.imwrite(output_path, (image * 255).astype(np.uint8))

# Path to the folder to save preprocessed images
output_folder = '/content/drive/MyDrive/Braille Dataset preprocessed'

# Save preprocessed images for each class
for class_name, images in images_per_class.items():
    class_output_folder = os.path.join(output_folder, class_name)
    os.makedirs(class_output_folder, exist_ok=True)
    save_preprocessed_images(images, class_output_folder)


In [ ]:
from sklearn.model_selection import train_test_split

# Function to split dataset into train, validation, and test sets
def split_dataset(images_per_class, test_size=0.2, validation_size=0.25, random_state=None):
    train_images = []
    train_labels = []
    validation_images = []
    validation_labels = []
    test_images = []
    test_labels = []

    for class_name, images in images_per_class.items():
        # Split images for each class
        X_train, X_test, y_train, y_test = train_test_split(images, [class_name]*len(images), test_size=test_size, random_state=random_state, stratify=[class_name]*len(images))
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_size/(1-test_size), random_state=random_state, stratify=y_train)

        # Add split images to respective sets
        train_images.extend(X_train)
        train_labels.extend(y_train)
        validation_images.extend(X_val)
        validation_labels.extend(y_val)
        test_images.extend(X_test)
        test_labels.extend(y_test)

    return train_images, validation_images, test_images, train_labels, validation_labels, test_labels

# Split dataset
train_images, validation_images, test_images, train_labels, validation_labels, test_labels = split_dataset(images_per_class)

# Print sizes of each set
print("Train set size:", len(train_images))
print("Validation set size:", len(validation_images))
print("Test set size:", len(test_images))

In [ ]:
# Define ResNet architecture
class BasicBlock(nn.Module):
    expansion = 1  # Set expansion attribute

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=26):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # Change input channels to 1
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self.make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self.make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self.make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self.make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def make_layer(self, block, out_channels, blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Define hyperparameters
learning_rate = 0.001
batch_size = 64
num_epochs = 10

# Initialize model with double precision parameters
model = ResNet(BasicBlock, [2, 2, 2, 2]).double()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Convert class names to numerical labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
validation_labels_encoded = label_encoder.transform(validation_labels)

# Convert data to PyTorch tensors and create DataLoader
train_dataset = TensorDataset(torch.tensor(train_images), torch.tensor(train_labels_encoded))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = TensorDataset(torch.tensor(validation_images), torch.tensor(validation_labels_encoded))
validation_loader = DataLoader(validation_dataset, batch_size=batch_size)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs = inputs.double()  # Convert inputs to double
        outputs = model(inputs.unsqueeze(1))  # Add an extra dimension for the single channel
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs = inputs.double()  # Convert inputs to double
            labels = torch.tensor(labels)  # Convert labels to tensor format
            outputs = model(inputs.unsqueeze(1))  # Add an extra dimension for the single channel
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

    # Print statistics
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_val_loss = val_loss / len(validation_loader.dataset)
    accuracy = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Validation Loss: {epoch_val_loss:.4f}, Accuracy: {accuracy:.2%}")

print('Finished Training')

In [ ]:
# Set model to evaluation mode
model.eval()

# Lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the validation set and make predictions
with torch.no_grad():
    for inputs, labels in validation_loader:
        inputs = inputs.double()  # Convert inputs to double
        labels = torch.tensor(labels)  # Convert labels to tensor format
        outputs = model(inputs.unsqueeze(1))  # Add an extra dimension for the single channel
        _, predicted = torch.max(outputs, 1)

        true_labels.extend(labels.numpy())
        predicted_labels.extend(predicted.numpy())

# Calculate precision, recall, and F1-score
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
# Initialize empty lists to store precision values and epochs
validation_precision = []
epochs = range(1, num_epochs + 1)

# Validation loop
for epoch in range(num_epochs):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs = inputs.double()  # Convert inputs to double
            labels = torch.tensor(labels)  # Convert labels to tensor format
            outputs = model(inputs.unsqueeze(1))  # Add an extra dimension for the single channel
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Calculate precision for this epoch
        precision = correct / total
        validation_precision.append(precision)

# Plot validation precision
plt.figure(figsize=(10, 6))
plt.plot(epochs, validation_precision, marker='o', color='blue')
plt.title('Validation Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.xticks(epochs)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Define classes
classes = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'
            'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
           'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
           'y', 'z']

# Calculate precision, recall, and f1-score for each class
precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(true_labels, predicted_labels, labels=classes)

# Plot precision, recall, and f1-score for each class
plt.figure(figsize=(10, 6))

plt.plot(classes, precision_per_class, marker='o', label='Precision')
plt.plot(classes, recall_per_class, marker='o', label='Recall')
plt.plot(classes, f1_per_class, marker='o', label='F1-score')

plt.title('Precision, Recall, and F1-score per class')
plt.xlabel('Class')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.show()

In [ ]:
# Define classes
classes = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'
            'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
           'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
           'y', 'z']

# Calculate precision for each class
precision_per_class, _, _, _ = precision_recall_fscore_support(true_labels, predicted_labels, labels=classes)

# Plot precision for each class
plt.figure(figsize=(10, 6))

plt.plot(classes, precision_per_class, marker='o', label='Precision', color='blue')

plt.title('Precision per class')
plt.xlabel('Class')
plt.ylabel('Precision')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()

In [ ]:

# Initialize empty lists to store precision and recall values for each epoch
validation_precision = []
validation_recall = []
epochs = range(1, num_epochs + 1)

# Validation loop
for epoch in range(num_epochs):
    model.eval()
    correct = 0
    total = 0
    true_positives = 0
    false_negatives = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs = inputs.double()  # Convert inputs to double
            labels = torch.tensor(labels)  # Convert labels to tensor format
            outputs = model(inputs.unsqueeze(1))  # Add an extra dimension for the single channel
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            true_positives += ((predicted == labels) & (labels == 1)).sum().item()
            false_negatives += ((predicted != labels) & (labels == 1)).sum().item()

        # Calculate precision and recall for this epoch
        precision = true_positives / (true_positives + (total - true_positives - false_negatives))
        recall = true_positives / (true_positives + false_negatives)
        validation_precision.append(precision)
        validation_recall.append(recall)

# Plot validation precision
plt.figure(figsize=(10, 6))
plt.plot(epochs, validation_precision, marker='o', color='blue')
plt.title('Validation Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.xticks(epochs)
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot validation recall
plt.figure(figsize=(10, 6))
plt.plot(epochs, validation_recall, marker='o', color='green')
plt.title('Validation Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.xticks(epochs)
plt.grid(True)
plt.tight_layout()
plt.show()

**Using pretrained resnet model**

In [ ]:
# Function to split dataset into train, validation, and test sets
def split_dataset(images_per_class, test_size=0.2, validation_size=0.25, random_state=None):
    train_images = []
    train_labels = []
    validation_images = []
    validation_labels = []
    test_images = []
    test_labels = []

    for class_name, images in images_per_class.items():
        # Split images for each class
        X_train, X_test, y_train, y_test = train_test_split(images, [class_name]*len(images), test_size=test_size, random_state=random_state, stratify=[class_name]*len(images))
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_size/(1-test_size), random_state=random_state, stratify=y_train)

        # Add split images to respective sets
        train_images.extend(X_train)
        train_labels.extend(y_train)
        validation_images.extend(X_val)
        validation_labels.extend(y_val)
        test_images.extend(X_test)
        test_labels.extend(y_test)

    return train_images, validation_images, test_images, train_labels, validation_labels, test_labels

# Split dataset
train_images, validation_images, test_images, train_labels, validation_labels, test_labels = split_dataset(images_per_class)

# Print sizes of each set
print("Train set size:", len(train_images))
print("Validation set size:", len(validation_images))
print("Test set size:", len(test_images))

In [ ]:
# Define transforms for data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean and std
])

# paths to our dataset directory
train_dir = 'path/to/train/dataset'
validation_dir = 'path/to/validation/dataset'

# Create datasets
train_dataset = torchvision.datasets.ImageFolder(root=train_dir, transform=transform)
validation_dataset = torchvision.datasets.ImageFolder(root=validation_dir, transform=transform)

# Define batch size
batch_size = 64

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size)

# Load pre-trained ResNet model
resnet = torchvision.models.resnet18(pretrained=True)

# Freeze the pre-trained parameters
for param in resnet.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes in your dataset
num_classes = 26  # Change this according to your dataset
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet.to(device)

# Training loop
num_epochs = 10  # Change this as needed
for epoch in range(num_epochs):
    resnet.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    # Validation
    resnet.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = resnet(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

    # Print statistics
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_val_loss = val_loss / len(validation_loader.dataset)
    accuracy = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Validation Loss: {epoch_val_loss:.4f}, Accuracy: {accuracy:.2%}")

print('Finished Training')